In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
import logging
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
!gsutil -m cp gs://tibot-ml-labeling/datasets/cats_vs_dogs_dataset.zip  ./

Copying gs://tibot-ml-labeling/datasets/cats_vs_dogs_dataset.zip...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

|
Operation completed over 1 objects/593.6 MiB.                                    


In [ ]:
!unzip -q /content/cats_vs_dogs_dataset.zip

In [ ]:
import os
import torch
import timm
import poutyne
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from sklearn.metrics import classification_report, confusion_matrix
import wandb
from poutyne.framework import Model, Callback
from google.cloud import storage

In [ ]:
# Check if a GPU is available and use it, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define the directory path
model_dir = '/content/models'

# Check if the directory exists, and create it if it doesn't
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [ ]:
#  Define dataset paths
train_dir = '/content/dataset/train'
test_dir = '/content/dataset/test'

# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Create custom datasets
train_dataset = ImageFolder(train_dir, transform=data_transforms['train'])
test_dataset = ImageFolder(test_dir, transform=data_transforms['test'])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# Define your model
model_name = 'tf_efficientnetv2_b2'
num_classes = 2  # Number of classes (cats and dogs)

# Load the model architecture and move it to the GPU
model = timm.create_model(model_name, pretrained=False, num_classes=num_classes).to(device)

In [ ]:
# Log in with your WANDB API key
wandb.login(key="1adf1a1fccd0e0da79c739f91daa90e0da69abb7")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
class UploadModelCallback(poutyne.Callback):
    def __init__(self, bucket_name, remote_dir, num_epochs=1):
        super().__init__()
        self.num_epochs = num_epochs
        self.bucket_name = bucket_name
        self.remote_dir = remote_dir
        self.local_dir = "/content/models"

        self.storage_client = storage.Client()

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) >= 1:  # Upload the model checkpoint after each epoch
            # Save the model checkpoint locally
            checkpoint_path = os.path.join(self.local_dir, f"EfficientNetV2B2_checkpoint-{epoch + 1:02d}.ckpt")
            self.model.save_weights(checkpoint_path)

            # Upload the model checkpoint to Google Cloud Storage
            remote_filepath = os.path.join(self.remote_dir, os.path.basename(checkpoint_path))
            bucket = self.storage_client.bucket(self.bucket_name)
            blob = bucket.blob(remote_filepath)
            blob.upload_from_filename(checkpoint_path)

            print(f"Model checkpoint saved in {checkpoint_path} and uploaded to {remote_filepath}")

In [ ]:
# Initialize wandb
wandb.init(project="cats-vs-dogs-pytorch")

acc,▁▁▅▆█
epoch,▁▃▅▆█
loss,██▅▃▁
time,▃█▃▁▄
val_acc,▅▇▁█▄
val_loss,▂█▂▃▁
acc,71.78432
epoch,5
loss,0.59261
time,186.194
val_acc,73.30938


In [ ]:
# Define WandbCallback
class WandbCallback(poutyne.Callback):
    def on_epoch_end(self, epoch, logs):
        # Log metrics to Wandb
        wandb.log(logs, step=epoch)

In [ ]:
# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# Create the Poutyne Model
poutyne_model = Model(model, optimizer, criterion, batch_metrics=['accuracy']).to(device)

# Train the model with upload callback
num_epochs = 10
bucket_name = "tibot-ml-labeling"
remote_dir = "models/cats_vs_dogs_task/sarim/EfficientNetV2B2_Not_pretrained_/model"

# Define the upload callback
upload_callback = UploadModelCallback(bucket_name, remote_dir, num_epochs)

# Modify the callbacks list to include the WandbCallback
callbacks = [upload_callback, WandbCallback()]

# Train the model with the updated callbacks list
poutyne_model.fit_generator(train_loader, test_loader, epochs=num_epochs, callbacks=callbacks)

Epoch:  1/10 Train steps: 563 Val steps: 148 3m7.53s loss: 0.820910 acc: 64.330183 val_loss: 1.019648 val_acc: 61.094675
Model checkpoint saved in /content/models/EfficientNetV2B2_checkpoint-02.pth and uploaded to models/cats_vs_dogs_task/sarim/EfficientNetV2B2_Not_pretrained_/model/EfficientNetV2B2_checkpoint-02.pth
Epoch:  2/10 Train steps: 563 Val steps: 148 3m8.77s loss: 0.678666 acc: 67.409672 val_loss: 0.831540 val_acc: 70.667794
Model checkpoint saved in /content/models/EfficientNetV2B2_checkpoint-03.pth and uploaded to models/cats_vs_dogs_task/sarim/EfficientNetV2B2_Not_pretrained_/model/EfficientNetV2B2_checkpoint-03.pth
Epoch:  3/10 Train steps: 563 Val steps: 148 3m8.64s loss: 0.663032 acc: 68.221234 val_loss: 0.823749 val_acc: 71.428571
Model checkpoint saved in /content/models/EfficientNetV2B2_checkpoint-04.pth and uploaded to models/cats_vs_dogs_task/sarim/EfficientNetV2B2_Not_pretrained_/model/EfficientNetV2B2_checkpoint-04.pth
Epoch:  4/10 Train steps: 563 Val steps: 14

[{'epoch': 1,
  'time': 187.52506872499998,
  'loss': 0.8209104235070225,
  'acc': 64.33018343693817,
  'val_loss': 1.0196481143783334,
  'val_acc': 61.09467456266221},
 {'epoch': 2,
  'time': 188.77442567399976,
  'loss': 0.6786655789749565,
  'acc': 67.4096720417186,
  'val_loss': 0.8315402082941438,
  'val_acc': 70.66779375116602},
 {'epoch': 3,
  'time': 188.63840757499975,
  'loss': 0.6630319650998839,
  'acc': 68.22123402059576,
  'val_loss': 0.8237494278639328,
  'val_acc': 71.42857144791901},
 {'epoch': 4,
  'time': 188.76890195299984,
  'loss': 0.6317076041557314,
  'acc': 70.11117287381879,
  'val_loss': 0.7408406508240978,
  'val_acc': 77.04987320371936},
 {'epoch': 5,
  'time': 188.7787981880001,
  'loss': 0.5976541504851972,
  'acc': 71.1450806011817,
  'val_loss': 0.7029640029419909,
  'val_acc': 76.69061707523245},
 {'epoch': 6,
  'time': 189.050871679,
  'loss': 0.5917591204671081,
  'acc': 71.61200667206879,
  'val_loss': 0.644877823852223,
  'val_acc': 76.986475076296

In [ ]:
# Calculate and print test accuracy
_, test_accuracy = poutyne_model.evaluate_generator(test_loader)
print(f"Test Accuracy: {test_accuracy * 1:.2f}%")

Test steps: 148 26.33s test_loss: 0.520585 test_acc: 80.431107                                
Test Accuracy: 80.43%


In [ ]:
import numpy as np
import torch
from sklearn.metrics import classification_report, confusion_matrix

# Initialize lists to store predictions and true labels
true_labels = []
predicted_labels = []

# Set the model to evaluation mode
model.eval()

# Move the model to the same device as the input data
model = model.to(device)

# Initialize lists to store predictions and true labels
true_labels = []
predicted_labels = []

# Iterate over the test dataset and collect predictions and true labels
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Get predicted class (the class with the highest probability)
        predicted = outputs.argmax(axis=1)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# Calculate and print the classification report
class_names = train_dataset.classes
classification_rep = classification_report(true_labels, predicted_labels, target_names=class_names)
print("Classification Report:")
print(classification_rep)

# Calculate and print the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("\nConfusion Matrix:")
print(conf_matrix)

Classification Report:
              precision    recall  f1-score   support

        cats       0.91      0.81      0.86      3505
        dogs       0.59      0.78      0.67      1227

    accuracy                           0.80      4732
   macro avg       0.75      0.80      0.77      4732
weighted avg       0.83      0.80      0.81      4732


Confusion Matrix:
[[2851  654]
 [ 272  955]]


In [ ]:
import torch.onnx
import torchvision.models as models

# Load your PyTorch model
model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)

# Set the model to evaluation mode
model.eval()

input_data = torch.randn(1, 3, 224, 224)  # Adjust the shape as per your model's input
onnx_path = "model.onnx"

# Export the PyTorch model to ONNX format
torch.onnx.export(model, input_data, onnx_path, verbose=True)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
import time

while True:
    print("Colab is still running...")
    time.sleep(60)

Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is still running...
Colab is sti

In [ ]:
# tunnel vpn
# run on fast vpn


import onnxruntime as ort
import numpy as np

# Load the ONNX model
onnx_model = ort.InferenceSession(onnx_path, providers=providers)

# Provide input data (change this according to your model's input)
input_data =
# Run inference
output = onnx_model.run(None, input_data)

SyntaxError: ignored